In [1]:
import geopandas as gpd
from shapely.geometry import LineString, Polygon
import matplotlib.pyplot as plt

In [2]:
gdf = gpd.read_file("Data/EXTEND_SDC_LEFT_M230905.shp")
gdf = gdf.to_crs(epsg=2056)  # Example: CH1903+ / LV95 for Switzerland
# Create a 10-meter buffer around each segment
gdf["buffer"] = gdf.geometry.buffer(10)
gdf.head()

/opt/anaconda3/envs/ESO_env/lib/python3.12/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


,mission,id,name,starttime,endtime,RMS,RANK,geometry,buffer
0,230905,1,M230905_196391_196440_LEFT.rxp,196391.045311,196440.132053,0.078102,39,"LINESTRING (2582689.195 1113865.115, 2582688.4...","POLYGON ((2582698.485 1113848.886, 2582697.798..."
1,230905,2,M230905_196440_196444_LEFT.rxp,196440.132338,196443.986397,0.073273,12,"LINESTRING (2582654.868 1113149.5, 2582652.235...","POLYGON ((2582662.018 1113134.996, 2582661.943..."
2,230905,3,M230905_197120_197122_LEFT.rxp,197119.898984,197121.910178,0.087459,80,"LINESTRING (2579891.232 1111037.035, 2579890.6...","POLYGON ((2579898.272 1111029.874, 2579898.209..."
3,230905,4,M230905_197128_197161_LEFT.rxp,197128.135028,197160.558478,0.087955,84,"LINESTRING (2579853.522 1111004.171, 2579851.3...","POLYGON ((2579842.788 1111006.755, 2579840.648..."
4,230905,5,M230905_197161_197173_LEFT.rxp,197160.558481,197172.978278,0.083066,54,"LINESTRING (2579739.028 1111173.235, 2579737.4...","POLYGON ((2579729.099 1111170.176, 2579726.321..."


In [3]:

# Compare pairs
records = []

for i in range(len(gdf)):
    for j in range(i + 1, len(gdf)):
        buf1 = gdf.loc[i, "buffer"]
        buf2 = gdf.loc[j, "buffer"]

        if buf1.intersects(buf2):
            overlap_poly = buf1.intersection(buf2)

            if not overlap_poly.is_empty:
                # Intersect original lines with the overlap polygon
                line1 = gdf.loc[i, "geometry"].intersection(overlap_poly)
                line2 = gdf.loc[j, "geometry"].intersection(overlap_poly)

                # Take union of both clipped lines (they may be on opposite sides of the polygon)
                shared_line = line1.union(line2)

                # Get total length of shared line(s)
                shared_length = shared_line.length if not shared_line.is_empty else 0

                records.append({
                    "id_1": gdf.loc[i, "id"],
                    "id_2": gdf.loc[j, "id"],
                    "overlap_geom": overlap_poly,
                    "shared_length_m": shared_length,
                    "overlap_area_m2": overlap_poly.area
                })

# Create GeoDataFrame
overlap_gdf = gpd.GeoDataFrame(records, geometry="overlap_geom", crs=gdf.crs)

# Filter short overlaps if you want (e.g. crossings)
# overlap_gdf = overlap_gdf[overlap_gdf["overlap_length_m"] > 10]

# Show info
print(overlap_gdf[["id_1", "id_2", "shared_length_m"]])

/opt/anaconda3/envs/ESO_env/lib/python3.12/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/ESO_env/lib/python3.12/site-packages/shapely/set_operations.py:131: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/opt/anaconda3/envs/ESO_env/lib/python3.12/site-packages/shapely/set_operations.py:334: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


     id_1  id_2  shared_length_m
0       1     2        19.982557
1       3    13        53.000608
2       4     5        19.992072
3       4     6        96.604351
4       4     7        59.328005
..    ...   ...              ...
480   320   321        55.197952
481   321   322        19.997094
482   322   323        19.992591
483   323   324        19.994578
484   324   325        20.005685

[485 rows x 3 columns]


In [ ]:

# Sort by shared length
overlap_gdf_sorted = overlap_gdf.sort_values(by="shared_length_m", ascending=True)

# Print relevant columns
print(overlap_gdf_sorted[["id_1", "id_2", "shared_length_m", "overlap_area_m2"]])
# %%


# how many segments intersect each zone
overlap_gdf["n_segments"] = overlap_gdf.geometry.apply(
    lambda geom: gdf["buffer"].intersects(geom).sum()
)

# Filter to only zones with ≥2 segments (true overlaps)
#overlap_nodes = overlap_gdf[overlap_gdf["n_segments"] >= 2].reset_index(drop=True)

# Centroid
overlap_gdf["centroid"] = overlap_gdf.geometry.centroid

# %%

overlap_gdf
# %%

most_segments_node = overlap_gdf.sort_values(by="n_segments", ascending=False)
most_segments_node
# %%
# Make sure your original gdf uses "id" as index (for fast lookup)
gdf_indexed = gdf.set_index("id")
#print(gdf_indexed)
# Now enrich overlap_gdf with time information
overlap_gdf["starttime_1"] = overlap_gdf["id_1"].map(gdf_indexed["starttime"])
overlap_gdf["endtime_1"] = overlap_gdf["id_1"].map(gdf_indexed["endtime"])

overlap_gdf["starttime_2"] = overlap_gdf["id_2"].map(gdf_indexed["starttime"])
overlap_gdf["endtime_2"] = overlap_gdf["id_2"].map(gdf_indexed["endtime"])
overlap_gdf.head()


     id_1  id_2  shared_length_m  overlap_area_m2
138   110   112         0.000000        41.845021
279   190   235         0.000000       497.846340
280   190   238         0.000000        75.956463
281   190   239         0.000000       179.101550
394   252   277         0.000000        52.141616
..    ...   ...              ...              ...
357   244   245       399.045886      3949.062796
316   207   214       433.493616      3807.283570
462   300   302       624.183917      4506.381000
278   190   234       760.972548      6151.322237
460   299   303      1624.299838     16148.551431

[485 rows x 4 columns]


/opt/anaconda3/envs/ESO_env/lib/python3.12/site-packages/shapely/predicates.py:778: RuntimeWarning:

invalid value encountered in intersects



,id_1,id_2,overlap_geom,shared_length_m,overlap_area_m2,n_segments,centroid,starttime_1,endtime_1,starttime_2,endtime_2
0,1,2,"POLYGON ((2582664.401 1113146.483, 2582664.059...",19.982557,313.571370,2,POINT (2582654.868 1113149.502),196391.045311,196440.132053,196440.132338,196443.986397
1,3,13,"POLYGON ((2579898.209 1111029.801, 2579892.654...",53.000608,638.965278,2,POINT (2579885.863 1111030.769),197119.898984,197121.910178,198008.632304,198060.217460
2,4,5,"POLYGON ((2579730.137 1111168.659, 2579729.731...",19.992072,313.654464,2,POINT (2579739.028 1111173.235),197128.135028,197160.558478,197160.558481,197172.978278
3,4,6,"POLYGON ((2579779.358 1111096.812, 2579773.131...",96.604351,983.681175,3,POINT (2579772.691 1111121.734),197128.135028,197160.558478,197216.283401,197220.324774
4,4,7,"POLYGON ((2579791.678 1111079.036, 2579785.506...",59.328005,599.217355,3,POINT (2579788.127 1111098.411),197128.135028,197160.558478,197220.324776,197247.992422


In [30]:
def classify_overlap(length):
    if length < 10:
        return "intersection"
    elif length < 50:
        return "partial_overlap"
    else:
        return "shared_road"

# Apply classification
overlap_gdf["classification"] = overlap_gdf["shared_length_m"].apply(classify_overlap)

In [32]:
most_segments_node = overlap_gdf.sort_values(by="n_segments", ascending=False)
most_segments_node

,id_1,id_2,overlap_geom,shared_length_m,overlap_area_m2,n_segments,centroid,starttime_1,endtime_1,starttime_2,endtime_2,classification
357,244,245,"POLYGON ((2582385.564 1103447.116, 2582386.667...",399.045886,3949.062796,14,POINT (2582454.149 1103396.06),216839.643730,216877.932303,216915.562081,216959.576813,shared_road
372,245,267,"POLYGON ((2582494.046 1103346.841, 2582493.857...",193.975406,2087.360008,9,POINT (2582534.716 1103333.2),216915.562081,216959.576813,217904.574956,217920.228491,shared_road
428,267,269,"POLYGON ((2582520.958 1103351.638, 2582521.01 ...",86.705932,896.638910,9,POINT (2582509.136 1103349.463),217904.574956,217920.228491,217996.791393,218026.957649,shared_road
361,244,267,"POLYGON ((2582503.906 1103368.786, 2582504.387...",93.957979,915.607222,9,POINT (2582510.289 1103346.672),216839.643730,216877.932303,217904.574956,217920.228491,shared_road
363,244,269,"POLYGON ((2582498.027 1103376.957, 2582501.139...",109.230508,1063.153802,9,POINT (2582502.81 1103354.683),216839.643730,216877.932303,217996.791393,218026.957649,shared_road
...,...,...,...,...,...,...,...,...,...,...,...,...
186,139,140,"POLYGON ((2578487.988 1105489.491, 2578488.325...",20.006285,313.654442,2,POINT (2578497.537 1105486.522),206340.385262,206400.179796,206400.179798,206460.222252,partial_overlap
183,138,139,"POLYGON ((2578175.445 1104962.513, 2578176.229...",20.000888,313.654467,2,POINT (2578181.844 1104954.828),206280.378813,206340.385260,206340.385262,206400.179796,partial_overlap
180,137,158,"POLYGON ((2578554.303 1104970.258, 2578549.867...",81.508554,785.137291,2,POINT (2578554.776 1104981.475),206220.612487,206280.378811,207959.961494,208020.451864,shared_road
177,137,138,"POLYGON ((2578285.606 1104931.092, 2578284.728...",20.078169,313.654492,2,POINT (2578289.632 1104940.245),206220.612487,206280.378811,206280.378813,206340.385260,partial_overlap


In [5]:
import geopandas as gpd
import plotly.graph_objects as go
from shapely.geometry import mapping

In [18]:

# Step 2: Create the Plotly figure
fig = go.Figure()

# Base layer: show all original segments for context (light gray)
for _, row in gdf.iterrows():
    geom = row.geometry
    if geom.geom_type == "LineString":
        x, y = list(geom.xy[0]), list(geom.xy[1])
        fig.add_trace(go.Scatter(
            x=x, y=y,
            mode="lines",
            line=dict(color="lightgray", width=1),
            hoverinfo="skip",
            showlegend=False,
        ))

In [8]:
top5 = overlap_gdf.sort_values("n_segments", ascending=False).head(5).reset_index(drop=True)

In [25]:

# Step 3: Create frames for each top overlap zone
frames = []

for i, row in top5.iterrows():
    zone_geom = row["overlap_geom"]
    ids = [row["id_1"], row["id_2"]]
    segs = gdf[gdf["id"].isin(ids)]

    frame_data = []

    # Red polygon for overlap zone
    if zone_geom.geom_type == "Polygon":
        x, y = list(zone_geom.exterior.xy[0]), list(zone_geom.exterior.xy[1])

        frame_data.append(go.Scatter(
            x=x, y=y,
            mode="lines",
            fill="toself",
            line=dict(color="red", width=3),
            name="Overlap Zone",
            hoverinfo="text",
            text=(
                f"<b>Overlap Zone {i+1}</b><br>"
                f"Segments: {row['id_1']} & {row['id_2']}<br>"
                f"Shared Length: {row['shared_length_m']:.1f} m<br>"
                f"Area: {row['overlap_area_m2']:.1f} m²<br>"
                f"Start 1: {row['starttime_1']:.2f} — End 1: {row['endtime_1']:.2f}<br>"
                f"Start 2: {row['starttime_2']:.2f} — End 2: {row['endtime_2']:.2f}"
            )
        ))
    # Blue lines for the two segments involved
    for _, seg in segs.iterrows():
        geom = seg.geometry
        if geom.geom_type == "LineString":
            list(geom.xy[0]), list(geom.xy[1])
        elif geom.geom_type == "MultiLineString":
            x, y = [], []
            for part in geom.geoms:
                xi, yi = part.xy
                x.extend(xi)
                y.extend(yi)
        else:
            continue

        frame_data.append(go.Scatter(
            x=x, y=y,
            mode="lines",
            line=dict(color="blue", width=2),
            name=f"Segment {seg['id']}",
            hoverinfo="text",
            text=(
                f"Segment ID: {seg['id']}<br>"
                f"Start: {seg['starttime']}<br>"
                f"End: {seg['endtime']}"
            ),
            showlegend=False
        ))

In [ ]:



frames.append(go.Frame(data=frame_data, name=str(i)))

# Step 4: Add the first frame’s data to the figure
fig.frames = frames
fig.add_traces(frames[0].data)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed